In [1]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 39.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [2]:
from llama_cpp import Llama
from transformers import AutoTokenizer

# 새로운 시도

### 질문 형식의 예시를 여러 개 보여주기

In [ ]:
model_id = 'Bllossom/llama-3-Korean-Bllossom-70B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
PROMPT = '''Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문들이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성하면 됩니다.
이 때 질문들은 제시문들 간의 연관성을 파악해 이를 물어보는 것입니다."

Format :
"(가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가) 싱가포르의 ‘태형’과 (나) 흥보전의 ‘매품’을 서로 비교하시오."
'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변세트를 총 3개 생성해주세요. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.

생성 형식은
["질문 1"]
["답변 1"]

["질문 2"]
["답변 2"]

["질문 3"]
["답변 3"]
으로 부탁드립니다.

질문과 답변 세트를 3개 생성해주세요.

이제 새로운 제시문 세트를 보여드리겠습니다.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-70B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
[제시문 (가)]의 내용을 바탕으로, 어떤 이유로 사람들은 유행을 따르고 싶어하는 심리적 동기를 설명해주세요.

답변 1:
유행은 사람들이 자신들 주변의 다른 사람들과 일치하고 싶어하는 욕구와, 그들만의 개성을 표현하고자 하는 욕구가 복합적으로 얽혀 있는 것입니다. 따라서, 사람들은 상대들에게 인정과 안도감을 느끼기 위해 유행을 따르고 싶어합니다. 이는 그들이 주변 사람들에게 인정받고, 그들만의 스타일이 받아들여지고, 그들만의 개성이 존중받는 것을 의미합니다.

질문 2:
[제시문 (나)]의 내용을 바탕으로, 어떤 이유로 사람들은 옛 것들을 따라가고, 동시대 사람들을 모방하려는 심리적 동기를 설명해주세요.

답변 2:
옛것을 따르는 경향은 사람들의 변화를 평가절하하고, 현재의 규범을 중요시 하려는 경향이 함께 동반됩니다. 사람들은 과거를 존경하지 않고, 지금은 지금이 가장 중요한 것으로 여겨집니다. 또한, 사람들은 같은 시대를 사는 사람들과 연결하고자 하며, 동시대 사람들을 모방하려는 경향을 가지고 있습니다. 이는 사람들이 현재와 미래에 대한 불안감을 해소하고, 안정감을 느끼는 것을 의미합니다.

질문 3:
[제시문 (다)]의 내용을 바탕으로, 어떤 이유로 사람들은 언어 사용에서도 세대 차이와 연관이 있으며, 이를 어떻게 설명할 수 있을까요?

답변 3:
언어 사용에서도 세대 차이와 연관이 있습니다. 이는 사회적인 존재로서의 역할을 통해 자연스럽게 발생하는데, 예를 들어, '엄마, 아빠'와 같은 말은 어린이들의 말이고, '자네, 댁'과 같은 단어는 어른들의 말이라는 것이 그러한 예입니다. 또한, 새로운 세대가 언어를 만들고 유행어를 만들기도 하는데, 이는 또 다른 세대 차이 중 하나인 젊은 세대의 특성을 대변합니다. 따라서, 언어 사용에서 세대 차이와 연관이 있는 것은 그들이 사회적인 존재로서의 역할을 통해 자연스럽게 발생하는 것입니다.


### 건님의 instruction과 결합

In [ ]:
qa_list = []

In [ ]:
PROMPT = '''
Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문들이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성합니다.
이 때 질문들은 제시문들 간의 연관성을 파악해 이를 물어보는 것입니다."

Format :
"(다)의 사례들을 (가)와 (나)에 비추어 평가하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가)와 (나)의 내용을 바탕으로 제시문 (다)의 ᄀ을 평가하시오.",
"제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가)와 (나)에서 작가가 갖추어야 할 자질들을 찾아 차이점과 공통점을 설명하시오."
'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변세트를 3개 생성해야 합니다. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.

생성 형식은
["질문 1"
"질문 2"
"질문 3"
"답변 1"
"답변 2"
"답변 3"]
으로 해야 합니다.

질문은 반드시 명령조로 생성되어야 합니다.

이제 새로운 제시문 세트를 보여드리겠습니다.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-70B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
qa_list.append(generated)

Fetching 6 files: 100% 6/6 [00:00<00:00, 1114.27it/s]
/content


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: MLP-KTLim/llama-3-Korean-Bllossom-70B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

### 영문입력

In [ ]:
qa_list = []

In [ ]:
PROMPT = '''
Role: "You are a teacher who runs a college entrance essay interview academy in South Korea."
Task: "Your main job is to generate example questions from a set of example statements.
I'm going to show you an example question. When you're given a new set of presentations, you figure out what they're about and then you create a similar set of questions.
At this time, the questions are to identify the relationship between the presentations and ask them."

Format :
"(다)의 사례들을 (가)와 (나)에 비추어 평가하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가)와 (나)의 내용을 바탕으로 제시문 (다)의 ᄀ을 평가하시오.",
"제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가)와 (나)에서 작가가 갖추어야 할 자질들을 찾아 차이점과 공통점을 설명하시오."
'''

In [ ]:
test_text = '''You need to look at the following 3 presentations and create 3 sets of questions and answers for them. Please delete all other unnecessary sentences.
All questions and answers should be in Korean, and concise, consisting of 1-2 sentences.
Each question should have a different form and content that should not overlap.
Please create three questions and three answers. As I said, when generating questions, the content should not imply or suggest the answers. When creating questions, it's important to consider the relationships between the contents of each prompt.
The question must be a commanding tone, and without honorifics.
Each question and answer should be related to all prompts.
Questions should be concise, and should not quote prompt.

The generation format is as follows:
["Question 1"]
"Question 2"
"Question 3"
"Answer 1"
"Answer 2"
[Answer 3]

Now, let me show you a new set of presentations.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-70B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.85,
    "temperature":0.4,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
qa_list.append(generated)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Here are three sets of questions and answers for the three presents.

[질문 1]
유행은 누구든 따라 할 수 있는 것이 아니라는 점에 대해 설명하십시오.

답변 1>
유행은 바로 그 대중성을 지닌 요소 때문에 어느 누구든 따라 할 수 있는 것입니다. 유행은 수많은 사람들 사이에서 어떤 행동이나 취향이 보통으로 여겨지는 경우를 의미하는데, 이 행동이나 취향이 이미 많은 사람들이 하고 있기 때문에 누구든지 쉽게 따라 할 수 있습니다. 따라서 유행이 모든 사람들에게 개방적인 것이 아니라는 것은 틀린 생각입니다.

[질문 2]
유행이 나타나면 왜 사람들은 그 유행을 따라 하게 되는지 설명하십시오.

답변 2>
유행은 사람들이 자신이나 주변 사람들과 차별화되도록 해주고, 동시에 안도감과 만족감을 제공합니다. 따라서 사람들은 유행을 따르기 위해 노력하고, 그것이 그들을 주변 사람들과 동일시하지 않고 차별화된 개인으로 인정받도록 도와줍니다. 또한 유행은 사람들 사이의 상호작용과 커뮤니케이션을 원활하게 해주므로, 사람들은 이를 통해 유대감을 형성하고 즐거움을 느낄 수 있습니다.

[질문 3]
어떤 경우에서는 유행이 더 이상 존재하지 않을까요?

답변 3>
유행이 더 이상 존재하지 않는 경우는 두 가지입니다. 첫째, 일부 개인은 모방을 두려워하고, themselves로 구별되고 싶다면 유행을 따른다는 경우가 있습니다. 둘째, 모든 개인이 제각각의 특수한 존재가 되고자 하는 욕구가 결여된 경우입니다. 즉, 사람들이 다양성을 존중하지 않고 모두 같은 행동과 옷을 선택한다면, 그 때부터는 유행이 존재하지 않게 됩니다.


In [ ]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

[질문 1]
제시문 (가)의 내용을 기반으로, 어떤 사람들은 유행을 따른다는 이유가 무엇일까요? 

답변 1
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려고 하는 심리적인 이유와, 다른 사람들로부터 인정받고 동화되기 위한 만족감을 얻기 위해서이다. 

[질문 2]
제시문 (나)의 내용을 기반으로, 어떤 사람들은 왜 새로운 것을 숭배하고 동시대 사람들을 모방하느냐? 

답변 2
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌어내는 주요한 원리이다. 이 원칙들은 선조의 유산을 평가절하하고 현재의 규범을 중요시하려는 성향이 함께 동반된다. 

[질문 3]
제시문 (다)의 내용을 기반으로, 왜 사람들은 시대에 따라 다른 언어를 사용하느냐? 

답변 3
사람은 사회적인 존재이기 때문에 신분이나 역할에 따라 행동양식이 달라지며, 그 행동양식 중 하나가 언어 사용이다. 따라서, 세대 변화나 사회적 변화에 따라 사용되는 언어도 변화하게 된다. 특히, 젊은 세대는 새로운 유행어와 신조어를 만들어내며 유행에 민감하고 변화를 쉽게 받아들이는 경향이 때문이다.
**************************************************
[질문 1]
제시문 (가)의 내용을 바탕으로, 유행이 왜 모든 개인을 유혹하는 것인지 묻습니다.

[답변 1]
유행은 바로 다른 사람들과 나를 구분할 수 있는 좋은 방법이 되기 때문입니다. 모든 개인이 제각각 특수한 존재가 되고 싶어하는 욕구와 동시에 모방을 회피하고자 하는 심리적인 요인이 결합되어 있기 때문입니다. 

[질문 2]
제시문 (나)의 내용을 바탕으로, 왜 사람들은 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시기를 특징으로 만들게 되었을까요?

[답변 2]
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌어내는 두 가지 중요한 원리입니다. 이 원소들은 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 육안으로 결합됩니다. 

[질문 3]
제

# 0624 모델 구현

필요한 것

- 질문 평가 metric

- 생성할 질문의 개수 다르게 input 받기

- 제시문은 통합으로 넣기

- 프롬프트와 질문 생성을 나눠서 볼 수 있도록

In [3]:
import pandas as pd
import random
qa_list = []
df_list = []
columns = ['질문 개수', '프롬프트', 'instruction', '생성 질문']

In [111]:
#num_of_question = int(input("원하는 질문의 개수를 입력하세요 : "))
num_of_question = random.randint(3,5)

### 연세대 2023 인문계 학종전형
new_presentation= '''
"유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

다른 관점에서는, 옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

한편,사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''

In [112]:
if num_of_question == 3 :
  format = '''
  ["질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"]'''
elif num_of_question == 4 :
  format = '''
  ["질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"]'''

else :
  format = '''
  ["질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"
  "질문 5"
  "답변 5"]'''

In [117]:
PROMPT = '''
Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성합니다.
모든 질문은 반드시 "~하시오."로 끝나야 합니다."

Format :
"환경정책을 수립할 때 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오.",
"글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오.",
"제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오.",
"글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."
'''

In [118]:
test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성해야 합니다. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.
모든 질문은 반드시 "~하시오."로 끝나야 합니다.

생성 형식은
{format}
으로 해야 합니다.

이제 새로운 제시문을 보여드리겠습니다.
{new_presentation}
                '''

In [119]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":1000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
print(f'질문 개수 : {num_of_question}')
qa_list.append(generated)
df_list.append([num_of_question, PROMPT, test_text, generated])

Fetching 6 files: 100% 6/6 [00:00<00:00, 6506.16it/s]
/content


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1: 제시문에서는 유행이 어떤 현상을 나타내며, 그 이유는 무엇인가요. 
답변 1: 제시문은 유행이 사람들의 행동과 취향에 대한 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시키는 매우 독특한 현상이라고 말합니다. 이는 사람들이 주변의 다른 사람들과 유사한 행동을 하고 싶어 하는安도감을 느끼려는 심리적인 이유와, 그들만의 개성을 드러내려고 하는 심리적 이유가 복합적으로 얽혀 있는 것입니다.

질문 2: 유행이 나타나면 어떤 현상이 일어나는가요?
답변 2: 유행이 나타난다면, 사람들은 서로의 동향을 따라하게 되고, 서로의 행동을 모방하게 됩니다. 이는 피렌체에서 1390년경 남성이 착용한 뚜렷한 유행이 없어졌던 것과 같은 예시를 통해 알 수 있습니다. 

질문 3: 왜 유행은 전체적으로 지배하지 않을까요?
답변 3: 유행은 복합적인 심리적 요인들을 작용시켜서 나타납니다. 따라서, 어떤 사람들은 유행에 따라 행동하고, 어떤 사람들은 그에 대해 거부감을 느끼기도 합니다. 이러한 이유로 인해, 유행은 모든 사람들을 지배하지는 않습니다.

질문 4: 왜 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌게 되었나요?
답변 4: 변화에 대한 애정과 동시대인들에 대한 모양은 인간의 심리적 특성상 항상 존재할 것입니다. 이러한 원리는 서구의 상류사회에서도 변화를 수용하는 열정에 의해 확산되었다는 예를 통해 알 수 있습니다.

질문 5: 언어에서 어떤 변화가 일어나면 어떤 영향을 끼치는가요?
답변 5: 인간의 사회적인 존재로서의 행동 양식 중 하나가 언어 사용입니다. 따라서, 새로운 언어 유행이나 신조어가 생겨나면, 그 변화는 쉽게 확산되어 나이나 세대별로 차이를 보일 수 있습니다. 이에 따라, 세대에 따른 언어 차이도 발생할 수 있습니다.
질문 개수 : 5


In [120]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

질문 1:
제시문에서 사용된 유대교와 관련된 내용으로, 이를 어떻게 처리해야 하는지 논의하시오.

답변 1:
제시에서 사용되는 유대인 관련된 내용은 주로 반유대주의와 유대인의 대우에 대한 문제 해결을 다루고 있습니다. 이를 해결하기 위해 어떤 접근 방법을 사용할 수 있는지에 대해 논의해보면,

질문 2:
제시에서 언급된 역사적 사건과 관련된 내용으로, 이를 어떻게 평가해야 하는지 논의하시오.

답변 2:
제시에서 언급된 역사적 사건들은 유럽 역사와 유대인 역사가 어떻게 상호작용하는지에 대한 질문을 일으킵니다. 이러한 역사적 사건과 이를 향한 접근 방식은 어떤 관점에서 볼 것인지에 따라 달라질 수 있습니다. 예를 들어, 현대의학에서 보는 시각이나 철학적인 관점을 적용하여 평가할 수도 있습니다.

질문 3:
제시에서 언급된 이야기는 어떤 영향을 미쳤는지에 대해 설명하시오.

답변 3:
제시에서 언급된 이야기는 독자에게 다양한 감정을 일으키며, 사회적 함축도 내포하고 있습니다. 예를 들어, 유대인 학살의 사실을 들으면 청각적인 인상을 받고, 그 사건에 대한 책임감을 느끼게 됩니다. 또한 이야기가 어떤 사회적 동기에 따른 행동을 유발하는지도 보여줍니다. 이러한 이야기의 영향은 독자의 인식과 가치관에 따라 다를 수 있습니다.
**************************************************
질문 1:
제시문에서 사용된 단어들을 분석하여 브라우닝과 골드하겐의 주장이 어떤 측면에서 서로 다른지를 밝혀주세요.

답변 1:
브라우닝은 사회적 관계를 통해 나약한 인간이 과도한 범죄에 관여할 수 있다는 것을 주장합니다. 그는 사회적 압박, 복종, 결정적 사건들의 영향력이 중요한 역할을 한다는 것을 강조합니다. 반면에, 골드하겐은 반유대주의라는 당시 독일 사회의 특수성을 핵심적인 원인이라고 주장합니다. 그는 반유대주의 믿음이 홀로코스트를 유발했다는 점을 강조하고 있습니다.

질문 2:
제시문에서 어떤 주장이 가장 강력하다고 생각되고, 그 이유는 무엇인가요?

In [110]:
df = pd.DataFrame(df_list, columns=columns)
df

,질문 개수,제시문,프롬프트,instruction,생성 질문
0,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...","질문 1:\n제시문에서 사용된 유대교와 관련된 내용으로, 이를 어떻게 처리해야 하는..."
1,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 사용된 단어들을 분석하여 브라우닝과 골드하겐의 주장이 어떤 ...
2,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...","질문 1:\n제시문에서 사용된 두 가지 시각에서 바라볼 때, 유대인 학살의 주요 원..."
3,5,\n인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n[시선으로부터의 윤리적성에 대해 설명하십시오.] \n\n답변 1:\n[...
4,3,"\n""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n[제시전에서 언급된 내용에 대해 설명하시오.] \n\n답변 1:\n[제...
5,4,"\n""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시전에서 말하는 화자들의 입장을 비판하여 구분하시오.\n\n답변 1:...
6,4,"\n""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시전에서 말하는 화자들의 연령층별 차이는 어떤 영향을 미칠까요? \n...
7,4,\n사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n 제공된 주어진 대사에서 사용되는 '믿음'과 '신뢰'의 의미를 설...
8,4,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 언급된 역사적 연구의 주요 이론 중 어떤 것이 유대인 학살의...
9,3,\n사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 말하는 화자들이 자신의 입장을 각각 어떻게 방어할 것인지 논...


In [ ]:
df.to_csv('0624 시도.csv', encoding = 'cp949')

### 새로운 제시문을 여러 개 준비해서 입력

In [3]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]Downloading 'tokenizer_config.json' to '.huggingface/download/tokenizer_config.json.6c407ae86d155b3c4100b980084c5c8105cec980.incomplete'

tokenizer_config.json: 100% 51.3k/51.3k [00:00<00:00, 25.6MB/s]
Download complete. Moving file to tokenizer_config.json

README.md: 100% 7.61k/7.61k [00:00<00:00, 41.9MB/s]
Download complete. Moving file to README.md

.gitattributes: 100% 1.59k/1.59k [00:00<00:00, 12.8MB/s]
Download complete. Moving file to .gitattributes
Fetching 6 files:  17% 1/6 [00:00<00:03,  1.35it/s]
tokenizer.json:   0% 0.00/10.1M [00:00<?, ?B/s]

special_tokens_map.json: 100% 507/507 [00:00<00:00, 3.86MB/s]
Download complete. Moving file to special_tokens_map.json


llama-3-Korean-Bllossom-8B-Q4_K_M.gguf:   0% 0.00/5.02G [00:00<?, ?B/s]

llama-3-Korean-Bllossom-8B-Q4_K_M.gguf:   0% 21.0M/5.02G [00:00<00:24, 208MB/s]

llama-3-Korean-Bllossom-8B-Q4_K_M.gguf:   1% 62.9M/5.02G [00:00<00:15, 316MB/s]

llama-3-Korean-Bllossom-8B-Q4_K_M.ggu

In [4]:
import pandas as pd
import random
qa_list = []
df_list = []
columns = ['질문 개수', '제시문', '프롬프트', 'instruction', '생성 질문']

In [46]:
### 연세대 2023 인문계 학종전형 p.20
presentation1 = '''
"유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다|.

다른 관점에서는, 옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

한편,사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''
###[문제 1] 제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그관점을 비교, 분석하시오. [50점]
###[문제 2] 제시문 (라)의 조사 결과를 설명하고, 그 결과를 바탕으로 제시문 (나)와(다)의 논지를 각각 평가하시오. [50점]

###2023 연세대 학종 추천형 p.17
presentation2 = '''
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로 나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다. 가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

이제부터 화자를 바꾸어 설명해보겠다. 서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.  그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.  이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

한편, 다른 관점에서 이야기 해보겠다. 도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다. 그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다. 그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는 비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'''
###제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와(나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오. (50점)
###[문제 2] 제시문 (라)의 조사 결과를 요약하고, 제시문 (나)와 (다)를 바탕으로 그 결과를 설명하시오. (50점)

presentation3 = '''
진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다.
역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

이제 다른 관점에서 이야기 해보겠다. 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

한편, 다른 화자의 발언은 다음과 같다. 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.

'''
###(가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
###[문제 2] (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.

###2024 서울 p.5
presentation4 = '''
사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발표를 준비하는 데에 있어 같은 반 친구 유진이가 당신에게 도움을 줄 것인가? 당신이 유진이가 과제를 도와줄 것이라는 사실을 믿는다면, 그것은 주변 친구들을 기꺼이 도와주었던 유진이의 평소 행동 등 증거에 바탕을 둔 것일 수 있다. 반면 과제를 도와줄 것이란 사실과 관련하여 당신이 유진이라는 사람을 믿는 것은 그와의 개인적 관계에 기반한다. 설령 유진이와 친하지 않더라도 당신은 유진이가 과제를 도와줄 것이란 사실을 믿을 수 있지만, 당신이 유진이를 믿는 것은 그에 대한 당신의 개인적 태도 없이는 성립할 수 없다. ‘믿음’을 사실에 대한 믿음에, ‘신뢰’를 사람에 대한 믿음에 한정해서 말한다면, 당신이 유진이가 과제를 도와줄 것이라고 ‘믿는’ 것과 유진이가 과제를 도와줄 것이라고 ‘신뢰하는’ 것은 같은 것이 아니다.

한편, 믿음의 기반과 신뢰의 기반의 차이는 믿었던 바가 참이 아닌 것으로 드러난 경우와 신뢰했던 바가 참이 아닌 것으로 드러난 경우에 나타나는 반응의 차이를 만든다. 오후 날씨가 맑을 것이라고 믿었지만 그렇지 않은 것으로 드러났다면, 실망스럽거나 짜증이 날 수 있다. 그리고 앞으로 날씨 예측과 관련해 더 많은 증거를 찾거나 다른 종류의 증거를 찾기도 할것이다. 반면, 절도 혐의를 받고 있는 친구가 결백을 호소하여 그가 결백하다고 신뢰했지만 그렇지 않은 것으로 드러난 경우, 우리가 느끼는 바는 단지 실망스러움이나 짜증이기보다는 배신감이다.

다른 관점에서 이야기하자면, 여행을 하다가 낯선 도시에 들러 식당을 찾아갈 때, 우리는 처음 보는 사람에게 길을 묻고 그가 일러 주는 방향으로 간다. 이때 우리는 그 사람이 어떤 사람인지 특별히 아는 바가 없고, 그가 잘 알지 못하면서 무책임하게 답했다거나 우리를 골탕 먹이기 위해 엉뚱한 방향을 알려주지 않았다는 사실을 아는 것도 아니다. 그럼에도 불구하고 그 낯선 이가 말해준 방향대로 길을 간다.

또한 다른 화자는 이렇게 말한다. 사람들이 서로를 잘 신뢰하는 사회에서는 타인의 말을 쉽게 믿어 버리고 타인의 말에 더 쉽게 속을 수도 있기 때문에 거짓이 팽배해질 수 있을 것이라 생각할 수 있다. 그러나 이 사회는 거짓이 배제되고 참이 증진되는 건강한 사회로 유지된다.

'''
###[문제 1] (가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.
###[문제 2] (가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.


new_presentation = [presentation1, presentation2, presentation3, presentation4]

In [47]:
#num_of_question = int(input("원하는 질문의 개수를 입력하세요 : "))
num_of_question = random.randint(3,5)

if num_of_question == 3 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"'''
elif num_of_question == 4 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"'''

else :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"
  "질문 5"
  "답변 5"'''

In [48]:
PROMPT = '''
당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.
["환경정책을 수립할 때 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오."]
["글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오."]
["제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."]
["글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."로 끝나야 합니다.
또한 제시문에서 사용된 단어를 사용하면 안 됩니다.
'''

In [49]:
presentation_number = random.randint(0,3)

test_text = f'''다음 제시문을 보고, 질문과 답변 세트 {num_of_question}개 생성하라. 다른 불필요한 문장은 생성하면 안 된다.
질문은 반드시 한국어로, 1~2문장으로 구성해야 한다.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 한다. 반드시 '제시문'이라는 단어를 사용하고, 다른 단어는 '대사', '제시전' 등의 단어는  사용해서는 안 된다.
모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다.

예를 들어, 질문은
###
"글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오."
"제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."
"글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."
###
등의 형식이어야 한다.

생성 형식은
{format}
으로 해야한다.

새로운 제시문
###
{new_presentation[presentation_number]}
###
                '''

In [50]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":2000,
    "stop":["<|eot_id|>"],
    "top_p":0.95,
    "temperature":0.5,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
print(f'질문 개수 : {num_of_question}')
qa_list.append(generated)
df_list.append([num_of_question, new_presentation[presentation_number], PROMPT, test_text, generated])

Fetching 6 files: 100% 6/6 [00:00<00:00, 6902.31it/s]
/content


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
제시문에서 언급된 문화적인 차이점에 대해 설명하십시오.

답변 1:
제시문에서 언급된 문화적인 차이점은 도시 문명과 과거 마을 사회의 차이입니다. 도시에서는 익명성이 전통에 따르는 도덕규범과 깊은 인간관계를 대신할 수 있도록 허용됩니다. 이는 도시의 익명성을 통해 개인이 자신을 자유롭게 발전시킬 수 있는 기회를 제공합니다.
질문 개수 : 5


In [51]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

질문 1:
제시문에서 사용된 유대교와 관련된 내용으로, 이를 어떻게 처리해야 하는지 논의하시오.

답변 1:
제시에서 사용되는 유대인 관련된 내용은 주로 반유대주의와 유대인의 대우에 대한 문제 해결을 다루고 있습니다. 이를 해결하기 위해 어떤 접근 방법을 사용할 수 있는지에 대해 논의해보면,

질문 2:
제시에서 언급된 역사적 사건과 관련된 내용으로, 이를 어떻게 평가해야 하는지 논의하시오.

답변 2:
제시에서 언급된 역사적 사건들은 유럽 역사와 유대인 역사가 어떻게 상호작용하는지에 대한 질문을 일으킵니다. 이러한 역사적 사건과 이를 향한 접근 방식은 어떤 관점에서 볼 것인지에 따라 달라질 수 있습니다. 예를 들어, 현대의학에서 보는 시각이나 철학적인 관점을 적용하여 평가할 수도 있습니다.

질문 3:
제시에서 언급된 이야기는 어떤 영향을 미쳤는지에 대해 설명하시오.

답변 3:
제시에서 언급된 이야기는 독자에게 다양한 감정을 일으키며, 사회적 함축도 내포하고 있습니다. 예를 들어, 유대인 학살의 사실을 들으면 청각적인 인상을 받고, 그 사건에 대한 책임감을 느끼게 됩니다. 또한 이야기가 어떤 사회적 동기에 따른 행동을 유발하는지도 보여줍니다. 이러한 이야기의 영향은 독자의 인식과 가치관에 따라 다를 수 있습니다.
**************************************************
질문 1:
제시문에서 사용된 단어들을 분석하여 브라우닝과 골드하겐의 주장이 어떤 측면에서 서로 다른지를 밝혀주세요.

답변 1:
브라우닝은 사회적 관계를 통해 나약한 인간이 과도한 범죄에 관여할 수 있다는 것을 주장합니다. 그는 사회적 압박, 복종, 결정적 사건들의 영향력이 중요한 역할을 한다는 것을 강조합니다. 반면에, 골드하겐은 반유대주의라는 당시 독일 사회의 특수성을 핵심적인 원인이라고 주장합니다. 그는 반유대주의 믿음이 홀로코스트를 유발했다는 점을 강조하고 있습니다.

질문 2:
제시문에서 어떤 주장이 가장 강력하다고 생각되고, 그 이유는 무엇인가요?

In [52]:
df = pd.DataFrame(df_list, columns=columns)
df


,질문 개수,제시문,프롬프트,instruction,생성 질문
0,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...","질문 1:\n제시문에서 사용된 유대교와 관련된 내용으로, 이를 어떻게 처리해야 하는..."
1,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 사용된 단어들을 분석하여 브라우닝과 골드하겐의 주장이 어떤 ...
2,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...","질문 1:\n제시문에서 사용된 두 가지 시각에서 바라볼 때, 유대인 학살의 주요 원..."
3,5,\n인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n[시선으로부터의 윤리적성에 대해 설명하십시오.] \n\n답변 1:\n[...
4,3,"\n""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n[제시전에서 언급된 내용에 대해 설명하시오.] \n\n답변 1:\n[제...
5,4,"\n""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시전에서 말하는 화자들의 입장을 비판하여 구분하시오.\n\n답변 1:...
6,4,"\n""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시전에서 말하는 화자들의 연령층별 차이는 어떤 영향을 미칠까요? \n...
7,4,\n사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n 제공된 주어진 대사에서 사용되는 '믿음'과 '신뢰'의 의미를 설...
8,4,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 언급된 역사적 연구의 주요 이론 중 어떤 것이 유대인 학살의...
9,3,\n사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 말하는 화자들이 자신의 입장을 각각 어떻게 방어할 것인지 논...


In [105]:
df.to_csv('0625 시도.csv', encoding = 'cp949')

UnicodeEncodeError: 'cp949' codec can't encode character '\u02dc' in position 3564: illegal multibyte sequence

### 다시 원래대로

In [53]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

Fetching 6 files: 100% 6/6 [00:00<00:00, 3520.19it/s]
/content


In [86]:
import pandas as pd
import random
qa_list_new = []
df_list_new = []
columns = ['질문 개수', '제시문', '프롬프트', 'instruction', '생성 질문']

In [87]:
### 연세대 2023 인문계 학종전형 p.20
presentation1 = '''
"유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다. 반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

다른 관점에서는, 옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다. 중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

한편,사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다. 이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''
###[문제 1] 제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그관점을 비교, 분석하시오. [50점]
###[문제 2] 제시문 (라)의 조사 결과를 설명하고, 그 결과를 바탕으로 제시문 (나)와(다)의 논지를 각각 평가하시오. [50점]

###2023 연세대 학종 추천형 p.17
presentation2 = '''
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로 나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다. 가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

이제부터 화자를 바꾸어 설명해보겠다. 서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.  그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.  이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

한편, 다른 관점에서 이야기 해보겠다. 도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다. 그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다. 그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는 비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'''
###제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와(나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오. (50점)
###[문제 2] 제시문 (라)의 조사 결과를 요약하고, 제시문 (나)와 (다)를 바탕으로 그 결과를 설명하시오. (50점)

###2024 서울 p.5
presentation3 = '''
사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발표를 준비하는 데에 있어 같은 반 친구 유진이가 당신에게 도움을 줄 것인가? 당신이 유진이가 과제를 도와줄 것이라는 사실을 믿는다면, 그것은 주변 친구들을 기꺼이 도와주었던 유진이의 평소 행동 등 증거에 바탕을 둔 것일 수 있다. 반면 과제를 도와줄 것이란 사실과 관련하여 당신이 유진이라는 사람을 믿는 것은 그와의 개인적 관계에 기반한다. 설령 유진이와 친하지 않더라도 당신은 유진이가 과제를 도와줄 것이란 사실을 믿을 수 있지만, 당신이 유진이를 믿는 것은 그에 대한 당신의 개인적 태도 없이는 성립할 수 없다. ‘믿음’을 사실에 대한 믿음에, ‘신뢰’를 사람에 대한 믿음에 한정해서 말한다면, 당신이 유진이가 과제를 도와줄 것이라고 ‘믿는’ 것과 유진이가 과제를 도와줄 것이라고 ‘신뢰하는’ 것은 같은 것이 아니다.

한편, 믿음의 기반과 신뢰의 기반의 차이는 믿었던 바가 참이 아닌 것으로 드러난 경우와 신뢰했던 바가 참이 아닌 것으로 드러난 경우에 나타나는 반응의 차이를 만든다. 오후 날씨가 맑을 것이라고 믿었지만 그렇지 않은 것으로 드러났다면, 실망스럽거나 짜증이 날 수 있다. 그리고 앞으로 날씨 예측과 관련해 더 많은 증거를 찾거나 다른 종류의 증거를 찾기도 할것이다. 반면, 절도 혐의를 받고 있는 친구가 결백을 호소하여 그가 결백하다고 신뢰했지만 그렇지 않은 것으로 드러난 경우, 우리가 느끼는 바는 단지 실망스러움이나 짜증이기보다는 배신감이다.

다른 관점에서 이야기하자면, 여행을 하다가 낯선 도시에 들러 식당을 찾아갈 때, 우리는 처음 보는 사람에게 길을 묻고 그가 일러 주는 방향으로 간다. 이때 우리는 그 사람이 어떤 사람인지 특별히 아는 바가 없고, 그가 잘 알지 못하면서 무책임하게 답했다거나 우리를 골탕 먹이기 위해 엉뚱한 방향을 알려주지 않았다는 사실을 아는 것도 아니다. 그럼에도 불구하고 그 낯선 이가 말해준 방향대로 길을 간다.

또한 다른 화자는 이렇게 말한다. 사람들이 서로를 잘 신뢰하는 사회에서는 타인의 말을 쉽게 믿어 버리고 타인의 말에 더 쉽게 속을 수도 있기 때문에 거짓이 팽배해질 수 있을 것이라 생각할 수 있다. 그러나 이 사회는 거짓이 배제되고 참이 증진되는 건강한 사회로 유지된다.

'''
###[문제 1] (가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.
###[문제 2] (가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.


new_presentation = [presentation1, presentation2, presentation3]

In [88]:
#num_of_question = int(input("원하는 질문의 개수를 입력하세요 : "))
num_of_question = random.randint(3,5)

if num_of_question == 3 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"'''
elif num_of_question == 4 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"'''

else :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"
  "질문 5"
  "답변 5"'''

In [100]:
PROMPT = '''
당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

예를 들어 다음과 같은 제시문 3개가 있습니다.

###
[가]진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[나]크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

[다]대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
###

이 제시문 세트에서 출제된 문제는 다음과 같습니다.
###
1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
2. (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.
###

이처럼 새로운 제시문이 주어지면 이에 대한 질문을 생성하면 됩니다.
이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.

["환경정책을 수립할 때 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오."]
["글에 등장하는 주장에 대 논하시오."]
["제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."]
["글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."로 끝나야 합니다.
또한 제시문에서 사용된 단어를 사용하면 안 됩니다.
'''

In [101]:
presentation_number = random.randint(0,2)

test_text = f'''다음 제시문을 보고, 질문과 답변 세트 {num_of_question}개 생성하라. 다른 불필요한 문장은 생성하면 안 된다.
질문은 반드시 한국어로, 1~2문장으로 구성해야 한다.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 한다. 반드시 '제시문'이라는 단어를 사용하고, 다른 단어는 '대사', '제시전' 등의 단어는  사용해서는 안 된다.
모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다.
또한 질문은 최대한 자연스러운 말투이어야 한다.

예를 들어, 질문은
###
"제시문에 기반하여 ~~~를 설명하시오."
"제시문을 통한 ~~~~에 대해 논의하시오."
"제시문에서 나타난 ~~~~에 대한 설명을 서술하시오."
###
등의 형식이어야 한다.

생성 형식은
{format}
으로 해야한다.

새로운 제시문
###
{new_presentation[presentation_number]}
###
                '''

In [102]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":2000,
    "stop":["<|eot_id|>"],
    "top_p":0.95,
    "temperature":0.5,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
print(f'질문 개수 : {num_of_question}')
qa_list_new.append(generated)
df_list_new.append([num_of_question, new_presentation[presentation_number], PROMPT, test_text, generated])

Fetching 6 files: 100% 6/6 [00:00<00:00, 4185.93it/s]
/content


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
제시문을 통해 제시된 주장에 대한 생각을 표현하시오.

답변 1:
제시문은 유행이 모든 분야에 영향을 미치는 현상으로 이야기하고 있습니다. 유행은 행동, 패션, 언어 등 다양한 분야에서 나타나며, 새로운 것이 항상 유행을 따라가는 경향이 있습니다. 이러한 현상은 모든 연령과 사회적 계층에서 동일하게 나타나는 것으로 보입니다.

질문 2:
제시문에서 언급된 것처럼 유대교인들도 강제수용 수용소에 수감되어 대량살해가 이루어졌습니다. 다른 학자들은 어떻게 이 문제에 접근했을까요?

답변 2:
다이너마이트 박사와 레오 카니오는 유태인의 학살에서 중요한 역할을 한 인물입니다. 데이나이트 박사는 히틀러의 반유대주의 정책을 지적하며 이를 비난했고, 카니오는 유태인으로서 자신이 겪은 고통을 책에 담아 유태인들의 이야기를 전파했습니다.

질문 3:
제시문에서 이야기된 것처럼 모든 사람들이 유행을 따라가는 것이 아니라는 점을 알려주시오.

답변 3:
유행에 대한 관심은 모든 연령과 사회적 계층에서 다릅니다. 노년층이나 보수층과 같은 전통적인 사람들은 변화를 꺼리며, 젊은 세대와 진보층을 따라가기 위해 노력하고 있습니다. 또한, 사람들마다 취미나 성향에 따라 유행에 대한 반응이 다를 수 있습니다.

질문 4:
제시문을 통해 어떤 유형의 동기가 사람들의 행동을 조작하는 역할을 하는지 설명하시오.

답변 4:
제시문은 사람들이 한 사회 집단의 일원임을 강조하기 위해 그들의 동기를 이야기합니다. 사람들은 자신이 속한 사회 집단의 가치관과 규범을 따라 행동하고, 이를 통해 사회적 안정과 통합이 이루어집니다. 하지만 때로는 이러한 동기가 사람들의 행동을 제한하고 제약하기도 합니다. 

질문 5:
제시문에서는 사람들이 유행을 어떻게 활용하여 자신의 개성과 구별되는 특징을 만들려고 하는지 설명하시오.

답변 5:
제시문은 사람들이 유행을 통해 자기 발명과 주변 사람들과 구별되는 특성을 만드는 데 큰 역할을 담당합니다. 사람들은 유행을 따르는 것보다는 자신이 속한 그룹이나 사회 집단의

In [103]:
for a in range(len(qa_list_new)):
  print(qa_list_new[a])
  print('*'*50)

질문 1:
제시문을 통해 사람들이 다른 사람들과 협력하기 위해 필요한 신뢰의 형태와 그 핵심 원리를 설명하시오.

답변 1:
사람들을 믿는 것과 사실을 믿는 것은 사뭇 다른 것이다. 협력을 위해 필요한 신뢰의 핵심 원리는 사람에 대한 신뢰가 있어야 한다는 것이다. 사람들은 자신이 알고 있는 사실의 상대적 중요성을 판단하지 못하고, 상대방의 말을 그대로 받아들이기 때문에, 어떤 일이 일어날지에 대한 추측을 통해 그 사람을 믿는다. 따라서, 동료들과 협력하기 위해서는 개인의 신뢰가 필수적이다.

질문 2:
제시문에서 언급된 것처럼, 역사가들은 어떤 역할을 수행하여 과거에 존재하는 사실을 재구성하는 데 중요한 역할을 한다는 것을 설명하시오.

답변 2:
역사학은 과거를 복원하고 이해하는 데 있어 중요한 역할을 수행하는 학문의 일종이다. 역사는 일어난 사건을 연구하고 기록하여, 그 이벤트의 세부사항과 이유를 재구성하는 것을 목표로 한다. 이를 통해 역사학자는 지난 시대들의 문화, 사회, 정치 등을 이해하고, 현재의 지식과 가치를 형성하는 데 기여한다. 따라서, 역사가들은 과거에 존재하는 사실들을 조사하고, 이를 바탕으로 과거를 복원하고 이해하는 데 중요한 역할을 수행한다.

질문 3:
제시문에서 언급된 것처럼, 허구성은 어떤 형태로 나타납니다? 예를 들어 설명하시오.

답변 3:
제시문에서 언급된 것처럼, 허구성은 일반적으로 이야기와 그 밖의 것을 결합시키는 합성적인 예술 형식이다. 이때 허구성을 통해 이야기는 사실과 함께 전달되며, 독자에게는 하나의 의미있는 세계를 제공합니다. 따라서, 허구성은 문학에서 이야기의 핵심 원리를 나타내는 중요한 요소 중 하나이다.

질문 4:
제시문을 통해 우리는 어떤 종류의 정보를 얻을 수 있는지 설명하시오.

답변 4:
제시문은 과거를 연구하는 데 유용한 정보를 제공한다. 이 정보는 역사적 사실을 재구성하고, 사회와 문화를 이해하는 데 도움을 줄 수 있다. 특히, 제시문은 독일 역사학자들이 수집한 자료를 바탕으로 하고 있으며, 이를

In [99]:
df_new = pd.DataFrame(df_list_new, columns=columns)
df_new


,질문 개수,제시문,프롬프트,instruction,생성 질문
0,5,\n사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문을 통해 사람들이 다른 사람들과 협력하기 위해 필요한 신뢰의 형태...
1,5,\n사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라. 다른 불필요한 문장은 생성하...","질문 1:\n제시문을 통해 제시된 내용과 관련된로, 어떤 사람들은 사소한 차이를 믿..."


In [76]:
df.to_csv('0625 시도_2.csv', encoding = 'cp949')